# Extract the spectrum of the z=16 candidate in the CEERS field

Source identified by Donnan et al. (2022) and  Naidu et al. (2022).

NIRSpec data from DD-2750 (PI: Pablo Arrabal Haro), expected 24 March 2023.

*Pipeline steps*

1. Run the preprocessing pipline through extracting 2D cutouts
1. Drizzle the cutouts to a combined spectrum
1. Extract 1D spectrum
1. Fit redshift

In [1]:
# Are we on a GitHub codespace?
import os

if os.getcwd().startswith('/workspaces/msaexp'):
    import os
    os.environ['CRDS_PATH'] = os.path.join('/tmp/', 'crds_cache')

    if not os.path.exists(os.environ['CRDS_PATH']):
        ! mkdir {os.environ['CRDS_PATH']}

    os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

    print('On codespace: ', os.environ['CRDS_PATH'], os.environ['CRDS_SERVER_URL'])

    workdir  = '/workspaces/msaexp/docs/examples/codespace'
    if not os.path.exists(workdir):
        ! mkdir {workdir}
    
    os.chdir(workdir)
else:
    print('(not on a codespace)')

In [2]:
import os
if 'CRDS_PATH' not in os.environ is None:
    os.environ['CRDS_PATH'] = f'{os.getcwd()}/crds_cache'

    if not os.path.exists(os.environ['CRDS_PATH']):
        os.makedirs(os.environ['CRDS_PATH'])

    os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

In [3]:
import os
import glob
import yaml
import warnings

import numpy as np
import matplotlib.pyplot as plt

import grizli
from grizli import utils, jwst_utils
jwst_utils.set_quiet_logging()
utils.set_warnings()

import astropy.io.fits as pyfits
from jwst.datamodels import SlitModel


import msaexp
from msaexp import pipeline

print(f'grizli version = {grizli.__version__}')
print(f'msaexp version = {msaexp.__version__}')

grizli version = 1.8.3
msaexp version = 0.6.3


# Query MAST for NIRSpec data

Query by program name and download `rate.fits` files with `mastquery`.  May need to set `$MAST_TOKEN` environment variable to enable the downloads from MAST.

Can optionally limit the query to specific 

 - gratings:  ``prism``, ``g140m``, ``g235m``, ``g395m``, ``g140m``, ``g235m``, ``g395m``
 - filters:  ``clear``, ``f170lp``, ``f100lp``, ``f170lp``, ``f290lp``
 - detectors: ``nrs1``, ``nrs2``
 
 

In [ ]:
# CEERS dropout
prog = 2750
gratings = ['prism']

outroot = 'ceers2750'

file_version = 'v1'
src_rd = utils.read_catalog("""# ra dec
214.914500 52.94304""")

# RXJ2129
prog = 2767
detectors = ['nrs2']
gratings = ['prism']

outroot = 'rxj2129'
src_rd = utils.read_catalog("""# ra dec
322.4215529 0.0916869""")

os.getcwd()

In [ ]:
import time

# Query NIRSpec data for a program name
files = glob.glob(f'jw0{prog}*rate.fits')

while len(files) == 0:
    print(time.ctime())
    masks = pipeline.query_program(prog, download=True, detectors=detectors, gratings=gratings)
    files = glob.glob(f'jw0{prog}*rate.fits')
    
    if len(files) == 0:
        time.sleep(30)

print(files)


In [ ]:
# Reprocess?

if grizli.__version__ >= 'xxxx1.8.3':
    from grizli import jwst_level1

    for file in files:
        _debug = jwst_level1.process_uncal_level1(file.replace('_rate','_uncal'),
                                         output_extension='_rate',
                                         jump_threshold=4,
                                         erode_snowballs=4,
                                         grow_snowballs=5,
                                         rescale_uncertainty=True,
                                        )

# Initialize pipeline

Exposures are grouped by detector and with a common `MSAMETFL` metadata file for the MSA setup.

In [ ]:
files = glob.glob(f'jw0{prog}*rate.fits')

groups = pipeline.exposure_groups(files=files)

print('\nFiles:\n======')
print(yaml.dump(dict(groups)))

## Preprocessing pipeline

1. Apply 1/f correction and identify "snowballs" on the `rate.fits` files
1. Remove "bias" (i.e., simple median) of each exposure
1. Rescale RNOISE array based on empty parts of the exposure
1. Run parts of the Level 2 JWST calibration pipeline ([calweb_spec2](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec2.html#calwebb-spec2)):
    - [AssignWcs](https://jwst-pipeline.readthedocs.io/en/latest/api/jwst.assign_wcs.AssignWcsStep.html) : initialize WCS and populate slit bounding_box data
    - [Extract2dStep](https://jwst-pipeline.readthedocs.io/en/latest/api/jwst.extract_2d.Extract2dStep.html) : identify slits and set slit WCS
    - [FlatFieldStep](https://jwst-pipeline.readthedocs.io/en/latest/api/jwst.flatfield.FlatFieldStep.html#flatfieldstep) : slit-level flat field
    - [PathLossStep](https://jwst-pipeline.readthedocs.io/en/latest/api/jwst.pathloss.PathLossStep.html) : NIRSpec path loss
    - [PhotomStep](https://jwst-pipeline.readthedocs.io/en/latest/api/jwst.photom.PhotomStep.html) : Photometric calibration
    - Note that the `srctype`, `master_background`, `wavecorr` steps are not performed.  The background subtraction is done manually on the 2D slit cutouts.
1. Parse slit metadata
1. Save slit cutout files of the last pipeline step performed (`phot` = `PhotomStep`)

Subsequent re-initialization of the `NirspecPipeline` object and `full_pipeline` method will skip these steps and just load the saved slit cutouts.

In [ ]:
for mode in groups:
    # Find the matching source
    pipe = pipeline.NirspecPipeline(mode=mode)

    idx, dr = src_rd.match_to_catalog_sky(pipe.msa.src_table)
    srcid = pipe.msa.src_table['source_id'][np.argmin(dr)]
    
    print(f'\n {mode} \n source_id: {srcid} dr={dr.min()}')

    # Run the pipeline to define slits, etc., but don't do extractions yet.  
    if not os.path.exists(f'{mode}.slits.yaml'):
        pipe = pipeline.NirspecPipeline(mode=mode,
                                        source_ids=[srcid],
                                       ) 
        
        pipe.full_pipeline(run_extractions=False,
                           initialize_bkg=False,
                           load_saved=None)
    else:
        print(f'Skip preprocessing: {pipe.mode}')

In [ ]:
files = glob.glob(f'{mode}*')
files.sort()
for f in files:
    print(f)

In [ ]:
print('Pipeline entries: ', list(pipe.pipe.keys()))
print('Last step: ', pipe.last_step)
print(pipe.mode)

## Extracted spectra 

The end products of the preprocessing pipeline are 2D calibrated spectra ([jwst.datamodels.SlitModel](https://jwst-pipeline.readthedocs.io/en/stable/api/jwst.datamodels.SlitModel.html)) for each slitlet.

In [ ]:
target = f'{srcid}'
#target = '11027' # z=9.5 Williams et al.
# target = '3593'

slit_files = glob.glob(f"*phot*_{target}.fits")

slit_files.sort()

slit_files

In [ ]:
dm = SlitModel(slit_files[0])

sh = dm.data.shape
fig, ax = plt.subplots(1,1,figsize=(8,8*sh[0]/sh[1]*2))
ax.imshow(dm.data, aspect='auto',
          cmap='plasma', vmin=-0.1, vmax=0.5)

In [ ]:
# From original detector image

exp = pyfits.open(f"{dm.meta.filename.split('_phot')[0]}_rate.fits")

slx = slice(dm.xstart, dm.xstart + dm.xsize)
sly = slice(dm.ystart, dm.ystart + dm.ysize)

fig, ax = plt.subplots(1,1,figsize=(8,8*sh[0]/sh[1]*2))
ax.imshow(exp['SCI'].data[sly, slx], aspect='auto',
          cmap='plasma', vmin=-0.1, vmax=0.5)

dm.close()

# Drizzle-combine 2D spectra

In [ ]:
from importlib import reload
import copy

import msaexp.drizzle
reload(msaexp.drizzle); reload(msaexp.utils)
reload(msaexp.drizzle); reload(msaexp.utils) 

DRIZZLE_PARAMS = copy.deepcopy(msaexp.drizzle.DRIZZLE_PARAMS)
DRIZZLE_PARAMS['kernel'] = 'square'
DRIZZLE_PARAMS['pixfrac'] = 1.0

_ = msaexp.drizzle.drizzle_slitlets(target,
                                    output=outroot,
                                    files=slit_files[:],
                                    center_on_source=False,  # Center output source as defined in MSA file.  Leave False and can maybe build master bkg
                                    fix_slope=0.2,          # Cross-dispersion scale dslit/dpix
                                    force_nypix=31,         # Y size of output array
                                    bkg_offset=6,           # Number of pixels to roll for background subtraction
                                    bkg_parity=[-1,1],      # Roll directions for background, i.e., -1 rolls the array down for bkg at top
                                    log_step=False,         # Log wavelength steps
                                    outlier_threshold=10,   # Outlier rejection threshold
                                    err_threshold=1000,     # Reject pixels in a slit where err > err_threshold*median(err)
                                    show_drizzled=True,     # Figures
                                    show_slits=True,
                                    imshow_kws=dict(vmin=-0.05, vmax=0.4, aspect='auto', cmap='cubehelix_r'),
                                    sn_threshold=-np.inf,   # Reject pixels where S/N < sn_threshold.  Prism S/N should usually be > 5 with bkg
                                    bar_threshold=0.0,      # Mask pixels where barshadow array less than this value
                                   )

figs, hdu_data, wavedata, all_slits, drz_data = _


# Optimal 1D extraction

In [ ]:
hdul = hdu_data['prism-clear']

outhdu = msaexp.drizzle.extract_from_hdul(hdul,
                                          prf_sigma=0.9, fix_sigma=False,
                                          prf_center=None, fix_center=False,
                                          verbose=True,
                                          )

outhdu.writeto(f'{outroot}_{target}.{file_version}.spec.fits', overwrite=True)

# Make figures
fig = msaexp.utils.drizzled_hdu_figure(outhdu, unit='fnu')
fig.savefig(f'{outroot}_{target}.{file_version}.fnu.png')

fig = msaexp.utils.drizzled_hdu_figure(outhdu, unit='flam')
fig.savefig(f'{outroot}_{target}.{file_version}.flam.png')


# Fit redshift

In [ ]:
import eazy

try:
    _ = templ # already defined
except:
    if not os.path.exists('templates'):
        eazy.symlink_eazy_inputs()

    otempl = eazy.templates.read_templates_file('templates/sfhz/blue_sfhz_13.param')

    if 1:
        wrest = utils.log_zgrid([300, 5.5e4], 1./500)
        print(wrest.shape)
        templ = []
        for i, t in enumerate(otempl):
            print(i, t.name)
            templ.append(t.smooth_velocity(500., in_place=False))
            templ[-1].resample(wrest, in_place=True)
    else:
        templ = otempl
    

In [ ]:
import msaexp.spectrum
import astropy.units as u

msaexp.spectrum.FFTSMOOTH = True
msaexp.spectrum.SCALE_UNCERTAINTY = 1.

file = f'{outroot}_{target}.{file_version}.spec.fits'

zfit_kwargs = dict(z0=[2.0, 17],
                   eazy_templates=templ,
                   vel_width=50,
                   scale_disp=1.0,
                   nspline=11,
                   Rline=2000,
                   use_full_dispersion=False,
                   is_prism=True,
                   sys_err=0.02,
                   ranges=((4600, 6800), (8800, 1.1e4)),
                  )

plt_kwargs = dict(eazy_templates=None,
                  vel_width=50,
                  scale_disp=1.3,
                  nspline=11,
                  Rline=2000,
                  use_full_dispersion=True,
                  is_prism=True,
                  sys_err=0.02,
                  ranges=((4600, 6800), (8800, 1.1e4)),
                  scale_uncertainty_kwargs={'order':1},
                  plot_unit=u.microJansky,
                  )

zfit = msaexp.spectrum.fit_redshift(file=file, **zfit_kwargs)



In [ ]:
reload(msaexp.spectrum)

best_z = zfit[1].meta['z']

if target in ['3349', '3593', '3314']:
    best_z = 10.17145

msaexp.spectrum.SCALE_UNCERTAINTY = 1.

_ = msaexp.spectrum.plot_spectrum(file=file,
                                  z=best_z,
                                  **plt_kwargs,
                                  )

In [ ]:
# Refit with just line templates

best_z = zfit[1].meta['z']

zfit_kwargs['z0'] = best_z + np.array([-0.1, 0.1])*(1+best_z)
zfit_kwargs['eazy_templates'] = None

zfit = msaexp.spectrum.fit_redshift(file=file, zstep=(0.0005, 0.0001), **zfit_kwargs)

best_z = zfit[1].meta['z']


In [ ]:
# Remake figures
fig = msaexp.utils.drizzled_hdu_figure(outhdu, unit='fnu', z=best_z)
fig.savefig(f'{outroot}_{target}.{file_version}.fnu.png')

fig = msaexp.utils.drizzled_hdu_figure(outhdu, unit='flam', z=best_z)
fig.savefig(f'{outroot}_{target}.{file_version}.flam.png')


# Extract all targets

In [ ]:
files = glob.glob(f'{mode}*')
files += glob.glob('*_phot*')
for file in files:
    os.remove(file)

In [ ]:
for mode in groups:
    # Run the pipeline to define slits, etc., but don't do extractions yet.  
    if True: #not os.path.exists(f'{mode}.slits.yaml'):
        pipe = pipeline.NirspecPipeline(mode=mode,
                                       ) 
        
        pipe.full_pipeline(run_extractions=False,
                           initialize_bkg=False,
                           load_saved=None)
    else:
        print(f'Skip preprocessing: {pipe.mode}')

In [ ]:
files = glob.glob('jw*phot*fits')

# files = glob.glob('jw01345070*nrs1*phot*1345_[0-9]*fits')

files.sort()


targets = [f.split('.')[-2].split('_')[-1] for f in files]
targets = np.unique(targets)

so = np.argsort(np.cast[int]([t.replace('m','-').replace('b','10000') for t in targets]))
targets = targets[so]
targets, len(targets)

In [ ]:
for t in targets:
    
    target = f'{t}'
    slit_files = glob.glob(f'*phot*_{t}.fits')
    slit_files.sort()
    
    if len(slit_files) == 0:
        continue
    
    try:
        
        _ = msaexp.drizzle.drizzle_slitlets(target,
                                        output=outroot,
                                        files=slit_files[:],
                                        center_on_source=False,  # Center output source as defined in MSA file.  Leave False and can maybe build master bkg
                                        fix_slope=0.2,          # Cross-dispersion scale dslit/dpix
                                        force_nypix=31,         # Y size of output array
                                        bkg_offset=6,           # Number of pixels to roll for background subtraction
                                        bkg_parity=[-1,1],      # Roll directions for background, i.e., -1 rolls the array down for bkg at top
                                        log_step=False,         # Log wavelength steps
                                        outlier_threshold=10,   # Outlier rejection threshold
                                        err_threshold=1000,     # Reject pixels in a slit where err > err_threshold*median(err)
                                        show_drizzled=True,     # Figures
                                        show_slits=True,
                                        imshow_kws=dict(vmin=-0.05, vmax=0.4, aspect='auto', cmap='cubehelix_r'),
                                        sn_threshold=-np.inf,   # Reject pixels where S/N < sn_threshold.  Prism S/N should usually be > 5 with bkg
                                        bar_threshold=0.0,      # Mask pixels where barshadow array less than this value
                                       )

        figs, hdu_data, wavedata, all_slits, drz_data = _
        
        hdul = hdu_data['prism-clear']

        outhdu = msaexp.drizzle.extract_from_hdul(hdul,
                                                  prf_sigma=0.9, fix_sigma=False,
                                                  prf_center=None, fix_center=False,
                                                  verbose=True,
                                                  )

        outhdu.writeto(f'{outroot}_{target}.{file_version}.spec.fits', overwrite=True)

        # Make figures
        fig = msaexp.utils.drizzled_hdu_figure(outhdu, unit='fnu')
        fig.savefig(f'{outroot}_{target}.{file_version}.fnu.png')

        fig = msaexp.utils.drizzled_hdu_figure(outhdu, unit='flam')
        fig.savefig(f'{outroot}_{target}.{file_version}.flam.png')

    except:
        continue


In [ ]:
# Break when running full notebook
print(break)

In [ ]:

def run_everything(target, skip=True):
    """
    Run all steps
    """
    
    if os.path.exists(f'{outroot}_{target}.{file_version}.flam.png') & skip:
        print(f'Skip: {target}')
        return True
    
    slit_files = glob.glob(f"*phot*_{target}.fits"))

    slit_files.sort()
    if len(slit_files) == 0:
        return False

    DRIZZLE_PARAMS = copy.deepcopy(msaexp.drizzle.DRIZZLE_PARAMS)
    DRIZZLE_PARAMS['kernel'] = 'square'
    DRIZZLE_PARAMS['pixfrac'] = 1.0

    _ = msaexp.drizzle.drizzle_slitlets(target,
                                        output=outroot,
                                        files=slit_files[:],
                                        center_on_source=False,  # Center output source as defined in MSA file.  Leave False and can maybe build master bkg
                                        fix_slope=0.2,          # Cross-dispersion scale dslit/dpix
                                        force_nypix=31,         # Y size of output array
                                        bkg_offset=6,           # Number of pixels to roll for background subtraction
                                        bkg_parity=[-1,1],      # Roll directions for background, i.e., -1 rolls the array down for bkg at top
                                        log_step=False,         # Log wavelength steps
                                        outlier_threshold=10,   # Outlier rejection threshold
                                        err_threshold=1000,     # Reject pixels in a slit where err > err_threshold*median(err)
                                        show_drizzled=True,     # Figures
                                        show_slits=True,
                                        imshow_kws=dict(vmin=-0.05, vmax=0.4, aspect='auto', cmap='cubehelix_r'),
                                        sn_threshold=-np.inf,   # Reject pixels where S/N < sn_threshold.  Prism S/N should usually be > 5 with bkg
                                        bar_threshold=0.0,      # Mask pixels where barshadow array less than this value
                                       )

    figs, hdu_data, wavedata, all_slits, drz_data = _

    # Optimal extraction
    hdul = hdu_data['prism-clear']

    outhdu = msaexp.drizzle.extract_from_hdul(hdul,
                                              prf_sigma=0.9, fix_sigma=False,
                                              prf_center=None, fix_center=False,
                                              verbose=True,
                                              )

    outhdu.writeto(f'{outroot}_{target}.{file_version}.spec.fits', overwrite=True)

    # Make figures
    fig = msaexp.utils.drizzled_hdu_figure(outhdu, unit='fnu')
    fig.savefig(f'{outroot}_{target}.{file_version}.fnu.png')

    fig = msaexp.utils.drizzled_hdu_figure(outhdu, unit='flam')
    fig.savefig(f'{outroot}_{target}.{file_version}.flam.png')

    # Fit redshift

    msaexp.spectrum.FFTSMOOTH = True

    file = f'{outroot}_{target}.{file_version}.spec.fits'

    zfit_kwargs = dict(z0=[1.0, 11],
                       eazy_templates=templ,
                       vel_width=50,
                       scale_disp=1.0,
                       nspline=11,
                       Rline=2000,
                       use_full_dispersion=False,
                       is_prism=True,
                       sys_err=0.02,
                       ranges=((4600, 6800), (8800, 1.1e4)),
                      )

    plt_kwargs = dict(eazy_templates=None,
                      vel_width=50,
                      scale_disp=1.0,
                      nspline=11,
                      Rline=2000,
                      use_full_dispersion=True,
                      is_prism=True,
                      sys_err=0.02,
                      ranges=((4600, 6800), (8800, 1.1e4)),
                      scale_uncertainty_kwargs={'order':4},
                      plot_unit=u.microJansky,
                      )

    zfit = msaexp.spectrum.fit_redshift(file=file, **zfit_kwargs)

    # Scale uncertainties
    best_z = zfit[1].meta['z']

    if target in ['3349', '3593', '3314']:
        best_z = 10.17145

    msaexp.spectrum.SCALE_UNCERTAINTY = 1.

    _ = msaexp.spectrum.plot_spectrum(file=file,
                                      z=best_z,
                                      **plt_kwargs,
                                      )

    #
    # Refit with just line templates

    best_z = zfit[1].meta['z']

    zfit_kwargs['z0'] = best_z + np.array([-0.1, 0.1])*(1+best_z)
    zfit_kwargs['eazy_templates'] = None

    zfit = msaexp.spectrum.fit_redshift(file=file, zstep=(0.0005, 0.0001), **zfit_kwargs)

    best_z = zfit[1].meta['z']
    
    # Remake figures
    fig = msaexp.utils.drizzled_hdu_figure(outhdu, unit='fnu', z=best_z)
    fig.savefig(f'{outroot}_{target}.{file_version}.fnu.png')

    fig = msaexp.utils.drizzled_hdu_figure(outhdu, unit='flam', z=best_z)
    fig.savefig(f'{outroot}_{target}.{file_version}.flam.png')
    
    return True


In [ ]:
for target in targets:
    try:
        run_everything(target)
    except:
        pass

    plt.close('all')